In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
def get_binary_address(given):
    global address_size
    
    format_str = "{0:0"+str(address_size)+"b}"
    res = format_str.format(int(given,16))
    return res

def get_block_offset(words):
    bo_size = int(math.log(words,2))
    #print(bo_size," bits")
    return bo_size

def get_index(words,word_size,cache_size):
    nr_lines = cache_size/(words*word_size)
    index_size = int(math.log(nr_lines,2))
    #print(index_size, "bits")
    return index_size

def get_tag(addr_sz,bo_sz,index_sz,byte_offset_sz):
    tag = addr_sz - index_sz - bo_sz
    #print(tag," bits")
    return tag

def calc_tag_index_offset(index_len,bo_len,addr):
       
    tag_len = len(addr) - index_len - bo_len
    tag = addr[:tag_len]
    index = addr[tag_len:tag_len+index_len]
    tag_index = tag + index
    offset = addr[tag_len+index_len:] + "00"
    #print("\nindex:",index,"\ntag:",tag)
    if(not index):
        return 0, int(tag,2), int(offset,2), int(tag_index,2)
    else:
        return int(index, 2), int(tag,2), int(offset,2), int(tag_index,2)

def get_cache_entry(given_addr):
    
    global byte_offset
    global block_offset
    
    #address tag and index conversion
    #print("\n\naddress:",given_addr)
    binary_addr = get_binary_address(given_addr)

    #binary address
    #print("full bit address:\n", binary_addr + "00")
    #print("word address:\n",binary_addr)

    #tag and index
    index_val,tag_val,offset_val,tag_index_val = calc_tag_index_offset(index,block_offset,binary_addr)
    #print("tag:",tag_val,"\nindex:",index_val)
    
    cache_entry = {"given_addr":given_addr,"word_addr":binary_addr,"tag":tag_val,\
                   "index":index_val,"offset":offset_val, "tag_index":tag_index_val*(2**(byte_offset + block_offset))}
    #print(cache_entry)
    
    return cache_entry


## Direct address format

In [3]:
#cache address format params
cache_type = 'Direct'

#address params (in bits)
byte_offset = 2
address_size = 32 - byte_offset

#word
word_size_bits = 32
word_size = word_size_bits/8

#cache
words_per_line = 4
cache_size = 2**10 * word_size


block_offset = get_block_offset(words_per_line)
index = get_index(words_per_line,word_size,cache_size)
tag = get_tag(address_size, block_offset, index, byte_offset)


#address format output
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset + byte_offset)
print("number of entries in cache:", cache_size / (words_per_line * word_size))

address format:
tag  20 	 index  8 	 block_offset 4
number of entries in cache: 256.0


## Set associative cache

In [4]:
#cache address format params
cache_type = 'Set associative'

#address params (in bits)
byte_offset = 2
address_size = 32 - byte_offset

#word
word_size_bits = 32 # 4 B
word_size = word_size_bits/8

#cache
words_per_line = 2**5
cache_size = 4096 * word_size
associativity = 4

nr_index = cache_size / (words_per_line * associativity * word_size)


block_offset = get_block_offset(words_per_line)
index = int(math.log(nr_index,2))
tag = get_tag(address_size, block_offset, index, byte_offset)


#address format output
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset + byte_offset)
print("sets:",nr_index)
print("number of entries in cache:", cache_size / (words_per_line * word_size))
print("number of entries in per set:", cache_size / (words_per_line * word_size * nr_index))

address format:
tag  20 	 index  5 	 block_offset 7
sets: 32.0
number of entries in cache: 128.0
number of entries in per set: 4.0


## Fully associative

In [5]:
#cache address format params
cache_type = 'Fully associative'
byte_offset = 2
address_size = 32 - byte_offset

#word
word_size_bits = 32
word_size = word_size_bits/8

#cache
words_per_line = 2
cache_size = 8 * word_size

block_offset = get_block_offset(words_per_line)
index = 0
tag = get_tag(address_size, block_offset, index, byte_offset)


#address format output
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset + byte_offset)

address format:
tag  29 	 index  0 	 block_offset 3


## Mapped cache address order from boot

In [6]:


addr_seq = '0x04 0x10 0x0c 0xb0 0xE4 0x9c 0x408 0x1c 0xb4 0xc1c 0x90 0x888'
addr_seq = addr_seq.split(" ")


cache = pd.DataFrame(columns = ["given_addr","word_addr","tag","index"])

for i in range(len(addr_seq)):
    
    #if decimal, uncomment
    #addr_seq[i] = hex(int(addr_seq[i]))
    
    
    cache_entry = get_cache_entry(addr_seq[i])
    cache_hit = cache[cache['index'] == cache_entry['index']]
    if(len(cache_hit)>0):
        cache_hit = cache_hit[cache_hit['tag'] == cache_entry['tag']]
        if(len(cache_hit)>0):
            cache_entry["H/M"] = 1
            cache_entry["replace"] = 0
        else:
            cache_entry["H/M"] = 0
            cache_entry["replace"] = 1
    else:
        cache_entry["H/M"] = 0
        cache_entry["replace"] = 0
        
    
    cache = cache.append(cache_entry,ignore_index=True)

    

hits = cache['H/M'].sum()
misses = len(cache) - hits
hit_rate = hits / len(cache)
replaces = cache['replace'].sum()

#print("hits / misses / hit_rate:",hits,"/",misses,"/",hit_rate)
print("HIT MISS CANNOT BE TRUSTED WITHOUT VALID BIT")
print("CONFIRM HIT MISS MANUALLY")

print("\n\nshowing for:",cache_type)
print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",block_offset + byte_offset)
cache

HIT MISS CANNOT BE TRUSTED WITHOUT VALID BIT
CONFIRM HIT MISS MANUALLY


showing for: Fully associative
address format:
tag  29 	 index  0 	 block_offset 3


,given_addr,word_addr,tag,index,H/M,offset,replace,tag_index
0,0x04,000000000000000000000000000100,2,0,0.0,0.0,0.0,16.0
1,0x10,000000000000000000000000010000,8,0,0.0,0.0,1.0,64.0
2,0x0c,000000000000000000000000001100,6,0,0.0,0.0,1.0,48.0
3,0xb0,000000000000000000000010110000,88,0,0.0,0.0,1.0,704.0
4,0xE4,000000000000000000000011100100,114,0,0.0,0.0,1.0,912.0
5,0x9c,000000000000000000000010011100,78,0,0.0,0.0,1.0,624.0
6,0x408,000000000000000000010000001000,516,0,0.0,0.0,1.0,4128.0
7,0x1c,000000000000000000000000011100,14,0,0.0,0.0,1.0,112.0
8,0xb4,000000000000000000000010110100,90,0,0.0,0.0,1.0,720.0
9,0xc1c,000000000000000000110000011100,1550,0,0.0,0.0,1.0,12400.0


## Cache specs

In [23]:
tag = 18
index = 7
offset = 7
byte_offset = 2

word_size = 32 #4B
nr_words_line = 2**(offset - byte_offset)
nr_cache_entries = 2**index
nr_words_cache = nr_words_line * nr_cache_entries

valid_bit = 1
lru_bit = 1

total_bits_per_entry_per_line = valid_bit + lru_bit + tag + nr_words_line*word_size #words to bits

print("address format:\ntag ",tag,"\t index ",index,"\t block_offset",offset)

print("\nno. of indexes:",nr_cache_entries,"\nno. of words in one line:",nr_words_line,"words")
print("no. of words in cache:",nr_words_cache,"words")

print("\ntotal width per cache entry (LRU+VALID+TAG+DATA):", total_bits_per_entry_per_line,"bits")
print("total data per line:", nr_words_line*word_size,"bits")

address format:
tag  18 	 index  7 	 block_offset 7

no. of indexes: 128 
no. of words in one line: 32 words
no. of words in cache: 4096 words

total width per cache entry (LRU+VALID+TAG+DATA): 1044 bits
total data per line: 1024 bits
